In [1]:
# Nationwide
import pandas as pd
import requests
from datetime import datetime, timedelta
from math import log10, floor

# Read the CSV into a pandas DataFrame, making sure to parse the first column as dates
# Specify the correct date format if pandas does not recognize it automatically
ww = pd.read_csv("US_Biobot_county_data_2024-04-29.csv", skiprows=2, parse_dates=['Date'], usecols=[1, 2, 5, 6], 
                 names=['County_FIPS', 'Date', 'Concentration', 'State_Abbrev'],
                 date_parser=lambda x: pd.to_datetime(x, format='%Y-%m-%d'))

# Turn County FIPS to int
ww['County_FIPS'] = ww['County_FIPS'].astype(int)

# Clean non-numeric characters from the concentration column if necessary
# For example, if there are commas in the numbers or there are strings like '<1'
ww['Concentration'] = pd.to_numeric(ww['Concentration'].replace('[^0-9.]', '', regex=True), errors='coerce')


# Read FIPS and population data
fips_pop_data = pd.read_csv('Fips_pop_short.csv')

# Prepare the population data
fips_pop_data['CENSUS_2020_POP'] = fips_pop_data['CENSUS_2020_POP'].fillna('0').str.replace(',', '').astype(int)

# Merge the wastewater data with the population data
merged_data = pd.merge(ww, fips_pop_data, left_on='County_FIPS', right_on='FIPStxt', how='left')

# Load the conversion factors CSV
conversion_factors_df = pd.read_csv('ConversionFactors/conversion_factors_by_state.csv')

# Create a dictionary mapping state abbreviations to conversion factors
conversion_factor_mapping = pd.Series(conversion_factors_df['Conversion Factor'].values, index=conversion_factors_df.State).to_dict()

def round_to_two_significant_digits(num):
    if num == 0:
        return 0
    else:
        round_digits = -int(floor(log10(abs(num)))) + 1
        return round(num, round_digits)

# Dictionary mapping state names to abbreviations
state_name_to_abbreviation = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'United States Virgin Islands': 'VI',
    'Vermont': 'VT',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}

rows_list = []
states = merged_data['State_Abbrev'].unique()

for state in states:
    # Filter for current state
    state_merged_data = merged_data[merged_data['State_Abbrev'] == state]

    # Calculate the weighted average for the current state
    state_weighted_avg_data = state_merged_data.groupby('Date').apply(lambda x: (x['Concentration'] * x['CENSUS_2020_POP']).sum() / x['CENSUS_2020_POP'].sum()).reset_index(name='weighted_avg_conc')
    state_weighted_avg_data['Date'] = pd.to_datetime(state_weighted_avg_data['Date'])
    state_weighted_avg_data = state_weighted_avg_data.sort_values('Date')

    # Prepare the wastewater data for the analysis period
    ww_state = state_weighted_avg_data.copy()
    if ww_state['Date'].size > 20:
        ww_state = ww_state.set_index('Date').resample('D').interpolate().reset_index()
        ww_state = ww_state[['Date', 'weighted_avg_conc']]
        ww_state['Date'] = pd.to_datetime(ww_state['Date'])
    else:
        # Use national average for comparison if there is not enough data
        ww_state = ww_state.set_index('Date').resample('D').interpolate().reset_index()
        ww_state = ww_state[['Date', 'weighted_avg_conc']]
        ww_state['Date'] = pd.to_datetime(ww_state['Date'])

        
    # Apply the conversion factor function
    ww_state['conversion_factor'] = conversion_factor_mapping[state]
    ww_state['estimated_infections'] = ww_state['weighted_avg_conc'] * ww_state['conversion_factor']
    
    # Apply the rounding function to the relevant columns
    ww_state['weighted_avg_conc'] = ww_state['weighted_avg_conc'].apply(round_to_two_significant_digits)

    for index, row in ww_state.iterrows():
        rows_list.append({'Country': 'United_States', 'Region': state, 'Date': row['Date'], 'Measure': 'inf', 'Value': row['estimated_infections']})
        rows_list.append({'Country': 'United_States', 'Region': state, 'Date': row['Date'], 'Measure': 'wastewater', 'Value': row['weighted_avg_conc']})

# Convert the combined DataFrame to JSON for use with ECharts
combined_df = pd.concat([pd.DataFrame(rows_list)], ignore_index=True)

combined_df.to_csv('US_states_min.csv', index=False)